In [155]:
import geopandas as gpd
import pandas as pd
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='pacaas-novas')

In [156]:
pacaas_area = gpd.read_file('C:/Users/ppgar/Downloads/QGIS/Pacaas_novos/pacaas_novas.geojson')

In [157]:
pacaas_area = pacaas_area.geometry.iloc[0].__geo_interface__

In [158]:
pacaas_area = ee.Geometry(pacaas_area)
buffer = pacaas_area.buffer(5000)
buffer_diff = buffer.difference(pacaas_area)

In [159]:
def calc_ndvi(image):
    nir = image.select('B8')
    red = image.select('B4')
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    return image.addBands(ndvi)

In [160]:
def mask_clouds(image):
    cloud_mask = image.select('QA60').eq(0)
    return image.updateMask(cloud_mask)

satelite_2016 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate('2016-05-01', '2016-09-30').map(mask_clouds).median().clip(buffer).select(['B11', 'B8', 'B4'])
satelite_2016_buffer = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate('2016-05-01', '2016-09-30').map(mask_clouds).median().clip(buffer_diff).select(['B11', 'B8', 'B4'])

In [161]:
def calc_evi(image):
    nir = image.select('B8')
    red = image.select('B4')
    blue = image.select('B2')
    evi = nir.subtract(red).divide(nir.add(red.multiply(6)).subtract(blue.multiply(7.5)).add(1)).multiply(2.5).rename('EVI')
    return image.addBands(evi)

In [162]:
pacaas_hist = pd.DataFrame(columns = ['Ano', 'Buffer_NDVI', 'Area_NDVI'])
for year in range(2016,2025):
    ndvi_mean = calc_ndvi(ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate(f'{year}-06-01', f'{year}-09-30').map(mask_clouds).median().clip(pacaas_area)).select(['NDVI']).reduceRegion('mean')
    ndvi_buffer = calc_ndvi(ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate(f'{year}-06-01', f'{year}-09-30').map(mask_clouds).median().clip(buffer_diff)).select(['NDVI']).reduceRegion('mean')
    aux = pd.DataFrame({'Ano': [year],
                        'Buffer_NDVI': [ndvi_buffer.get('NDVI').getInfo()],
                        'Area_NDVI': [ndvi_mean.get('NDVI').getInfo()]})
    pacaas_hist = pd.concat([pacaas_hist, aux])

,Ano,Buffer_NDVI,Area_NDVI
0,2016,0.709488,0.708851


,Ano,Buffer_NDVI,Area_NDVI
0,2017,0.699647,0.70018


,Ano,Buffer_NDVI,Area_NDVI
0,2018,0.721418,0.720925


,Ano,Buffer_NDVI,Area_NDVI
0,2019,0.635114,0.635198


,Ano,Buffer_NDVI,Area_NDVI
0,2020,0.703132,0.702197


,Ano,Buffer_NDVI,Area_NDVI
0,2021,0.66078,0.655374


,Ano,Buffer_NDVI,Area_NDVI
0,2022,0.629297,0.629308


,Ano,Buffer_NDVI,Area_NDVI
0,2023,0.680261,0.681938


,Ano,Buffer_NDVI,Area_NDVI
0,2024,0.629705,0.620735


In [163]:
pacaas_hist


,Ano,Buffer_NDVI,Area_NDVI
0,2016,0.709488,0.708851
0,2017,0.699647,0.700180
0,2018,0.721418,0.720925
0,2019,0.635114,0.635198
0,2020,0.703132,0.702197
0,2021,0.660780,0.655374
0,2022,0.629297,0.629308
0,2023,0.680261,0.681938
0,2024,0.629705,0.620735


In [164]:
ndvi = calc_ndvi(ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate('2023-06-01', '2023-08-30').map(mask_clouds).median().clip(buffer)).select(['NDVI'])
map =  geemap.Map()
map.addLayer(ndvi)
map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…